In [1]:
#dependencies
import pandas as pd
import numpy as np
import datetime as dt
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import scipy.stats as stats

# Extract & Transform

In [2]:
#store djia market data from CSV as a dataframe
file_dji = './Resources/DJI.csv'
dji_df = pd.read_csv(file_dji)
dji_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-01-03,11501.849609,11522.009766,11305.690430,11357.509766,11357.509766,169750000
1,2000-01-04,11349.750000,11350.059570,10986.450195,10997.929688,10997.929688,178420000
2,2000-01-05,10989.370117,11215.099609,10938.669922,11122.650391,11122.650391,203190000
3,2000-01-06,11113.370117,11313.450195,11098.450195,11253.259766,11253.259766,176550000
4,2000-01-07,11247.059570,11528.139648,11239.919922,11522.559570,11522.559570,184900000


In [3]:
#transform data as-needed
dji_df['DATE'] = pd.to_datetime(dji_df['Date'])
dji_df = dji_df.set_index('DATE')

#exception handle for re-runs
try:
    del dji_df['Date']
    del dji_df['Open']
    del dji_df['High']
    del dji_df['Low']
    del dji_df['Adj Close']
    del dji_df['Volume']
except KeyError:
    print('One or more keys not available for deletion (potential re-run)')
    pass

dji_df = dji_df.rename(columns={'Close':"Close_DJIA"})
dji_df.head()

,Close_DJIA
DATE,
2000-01-03,11357.509766
2000-01-04,10997.929688
2000-01-05,11122.650391
2000-01-06,11253.259766
2000-01-07,11522.559570


In [4]:
#store SARS data from CSV as a dataframe
file_sars = './Resources/sars_2003_complete_dataset_clean.csv'
sars_df = pd.read_csv(file_sars)
sars_df.head()

,Date,Country,Cumulative number of case(s),Number of deaths,Number recovered
0,2003-03-17,Germany,1,0,0
1,2003-03-17,Canada,8,2,0
2,2003-03-17,Singapore,20,0,0
3,2003-03-17,"Hong Kong SAR, China",95,1,0
4,2003-03-17,Switzerland,2,0,0


In [5]:
#transform data as-needed
del sars_df['Cumulative number of case(s)']
del sars_df['Number recovered']
sars_grouped_df = pd.DataFrame(sars_df.groupby('Date').sum())
sars_grouped_df.head()

,Number of deaths
Date,
2003-03-17,4
2003-03-18,4
2003-03-19,9
2003-03-20,10
2003-03-21,10


In [6]:
#store H1N1 data from CSV as a dataframe
file_h1n1 = './Resources/Pandemic (H1N1) 2009.csv'
h1n1_df = pd.read_csv(file_h1n1, encoding = 'latin1')
h1n1_df.head()

,Country,Cases,Deaths,Update Time
0,Algeria,5,0.0,7/6/2009 9:00
1,Antigua and Barbuda,2,0.0,7/6/2009 9:00
2,Argentina,2485,60.0,7/6/2009 9:00
3,Australia,5298,10.0,7/6/2009 9:00
4,Austria,19,0.0,7/6/2009 9:00


In [7]:
#transform data as-needed
h1n1_df['DateTime'] = pd.to_datetime(h1n1_df['Update Time'])
h1n1_df['Date'] = h1n1_df['DateTime'].dt.date
h1n1_groupby_df = pd.DataFrame(h1n1_df.groupby('Date').sum())
del h1n1_groupby_df['Cases']
h1n1_groupby_df.head()

,Deaths
Date,
2009-05-23,172.0
2009-05-25,182.0
2009-05-26,184.0
2009-05-27,190.0
2009-05-29,198.0


In [8]:
#store Ebola data from CSV as a dataframe
file_ebola = './Resources/ebola_2014_2016_clean.csv'
ebola_df = pd.read_csv(file_ebola)
ebola_df.head()

,Country,Date,No. of suspected cases,No. of probable cases,No. of confirmed cases,"No. of confirmed, probable and suspected cases",No. of suspected deaths,No. of probable deaths,No. of confirmed deaths,"No. of confirmed, probable and suspected deaths"
0,Guinea,2014-08-29,25.0,141.0,482.0,648.0,2.0,141.0,287.0,430.0
1,Nigeria,2014-08-29,3.0,1.0,15.0,19.0,0.0,1.0,6.0,7.0
2,Sierra Leone,2014-08-29,54.0,37.0,935.0,1026.0,8.0,34.0,380.0,422.0
3,Liberia,2014-08-29,382.0,674.0,322.0,1378.0,168.0,301.0,225.0,694.0
4,Sierra Leone,2014-09-05,78.0,37.0,1146.0,1261.0,11.0,37.0,443.0,491.0


In [9]:
#transform data as-needed
ebola_df = pd.DataFrame(ebola_df.groupby('Date').sum())
del ebola_df['No. of suspected cases']
del ebola_df['No. of probable cases']
del ebola_df['No. of confirmed cases']
del ebola_df['No. of confirmed, probable and suspected cases']
del ebola_df['No. of suspected deaths']
del ebola_df['No. of probable deaths']
del ebola_df['No. of confirmed, probable and suspected deaths']
ebola_df

,No. of confirmed deaths
Date,
2014-08-29,898.0
2014-09-05,1243.0
2014-09-08,1376.0
2014-09-12,1386.0
2014-09-16,1513.0
...,...
2015-12-17,5683.0
2015-12-22,5683.0
2015-12-23,5683.0


In [10]:
#store Covid-19 data from CSV as a dataframe
file_covid = './Resources/covid_19_clean_complete.csv'
covid_df = pd.read_csv(file_covid)
covid_df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Anhui,Mainland China,31.8257,117.2264,1/22/20,1,0,0
1,Beijing,Mainland China,40.1824,116.4142,1/22/20,14,0,0
2,Chongqing,Mainland China,30.0572,107.8740,1/22/20,6,0,0
3,Fujian,Mainland China,26.0789,117.9874,1/22/20,1,0,0
4,Gansu,Mainland China,36.0611,103.8343,1/22/20,0,0,0


In [11]:
#transform data as-needed
covid_df = pd.DataFrame(covid_df.groupby('Date').sum())
del covid_df['Lat']
del covid_df['Long']
del covid_df['Confirmed']
del covid_df['Recovered']
covid_df

,Deaths
Date,
1/22/20,17
1/23/20,18
1/24/20,26
1/25/20,42
1/26/20,56
1/27/20,82
1/28/20,131
1/29/20,133
1/30/20,171


In [12]:
#define function to create dataframe from market data and virus datsets post-clean

def pathogenify(marketdf, virusdf):
    #restrict market data date range to date range virus data applies to
    marketdf = marketdf.loc[virusdf.index[0]:virusdf.index[-1]]
    #merge data on outer join after date range restricted
    market_virus_df = pd.merge(marketdf,virusdf,left_index=True, right_index=True, how='outer')
    #rename columns
    market_virus_df.columns = ['Market','Deaths']
    #forward-fill NAs using prior day data if not NaN
    market_virus_df.fillna(method='ffill', inplace=True)
    #calculate daily percent change in market value
    market_virus_df['Pct_Chng_Market'] = market_virus_df['Market'].pct_change()*100
    #calculate daily percent change in deaths
    market_virus_df['Pct_Chng_Death'] = market_virus_df['Deaths'].pct_change()*100
    return market_virus_df

In [13]:
#use pathogenify function on sars + DJI data to form new dataframe
sars_market_df = pathogenify(dji_df,sars_grouped_df)
sars_market_df

,Market,Deaths,Pct_Chng_Market,Pct_Chng_Death
2003-03-17,8141.919922,4,NaN,NaN
2003-03-18,8194.230469,4,0.642484,0.000000
2003-03-19,8265.450195,9,0.869145,125.000000
2003-03-20,8286.599609,10,0.255877,11.111111
2003-03-21,8521.969727,10,2.840370,0.000000
...,...,...,...,...
2003-07-07,9216.790039,812,1.616060,0.000000
2003-07-08,9223.089844,812,0.068351,0.000000
2003-07-09,9156.209961,812,-0.725135,0.000000
2003-07-10,9036.040039,812,-1.312442,0.000000


In [14]:
#use pathogenify function on H1N1 + DJI data to form new dataframe
h1n1_market_df = pathogenify(dji_df,h1n1_groupby_df)
h1n1_market_df

,Market,Deaths,Pct_Chng_Market,Pct_Chng_Death
2009-05-23,NaN,172.0,NaN,NaN
2009-05-25,NaN,182.0,NaN,5.813953
2009-05-26,8473.490234,184.0,NaN,1.098901
2009-05-27,8300.019531,190.0,-2.047217,3.260870
2009-05-28,8403.799805,190.0,1.250362,0.000000
2009-05-29,8500.330078,198.0,1.148650,4.210526
2009-06-01,8721.440430,230.0,2.601197,16.161616
2009-06-02,8740.870117,230.0,0.222781,0.000000
2009-06-03,8675.240234,234.0,-0.750839,1.739130
2009-06-04,8750.240234,234.0,0.864529,0.000000


In [15]:
#use pathogenify function on H1N1 + DJI data to form new dataframe
ebola_market_df = pathogenify(dji_df,ebola_df)
ebola_market_df

,Market,Deaths,Pct_Chng_Market,Pct_Chng_Death
2014-08-29,17098.449219,898.0,NaN,NaN
2014-09-02,17067.560547,898.0,-0.180652,0.000000
2014-09-03,17078.279297,898.0,0.062802,0.000000
2014-09-04,17069.580078,898.0,-0.050937,0.000000
2014-09-05,17137.359375,1243.0,0.397077,38.418708
...,...,...,...,...
2016-03-17,17481.490234,5683.0,0.898838,0.000000
2016-03-18,17602.300781,5683.0,0.691077,0.000000
2016-03-21,17623.869141,5683.0,0.122531,0.000000
2016-03-22,17582.570313,5683.0,-0.234335,0.000000


In [16]:
#use pathogenify function on H1N1 + DJI data to form new dataframe
covid_market_df = pathogenify(dji_df,covid_df)
covid_market_df

,Market,Deaths,Pct_Chng_Market,Pct_Chng_Death
2020-01-22,29186.269531,17,NaN,NaN
2020-01-23,29160.089844,18,-0.089699,5.882353
2020-01-24,28989.730469,26,-0.584221,44.444444
2020-01-25,28989.730469,42,0.000000,61.538462
2020-01-26,28989.730469,56,0.000000,33.333333
2020-01-27,28535.800781,82,-1.565829,46.428571
2020-01-28,28722.849609,131,0.655488,59.756098
2020-01-29,28734.449219,133,0.040385,1.526718
2020-01-30,28859.439453,171,0.434984,28.571429
2020-01-31,28256.029297,213,-2.090859,24.561404


# LOAD

In [17]:
#connect to AWS cloud database
rds_connection_string = "postgres:postgres*@aws-bootcamp-server.cgk0xpvhfuev.us-east-2.rds.amazonaws.com:5432/postgres"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [18]:
#use pandas to load SARS data df into database
sars_market_df.to_sql(name = 'sars_market_data', con = engine, index_label = 'Date', if_exists = 'replace')
#query tables to validate load
pd.read_sql_query('select * from sars_market_data', con=engine).head()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"
FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
#use pandas to load H1N1 data df into database
h1n1_market_df.to_sql(name = 'h1n1_market_data', con = engine, index_label = 'Date', if_exists = 'replace')
#query tables to validate load
pd.read_sql_query('select * from h1n1_market_data', con=engine).head()

In [ ]:
#use pandas to load Ebola data df into database
ebola_market_df.to_sql(name = 'ebola_market_data', con = engine, index_label = 'Date', if_exists = 'replace')
#query tables to validate load
pd.read_sql_query('select * from ebola_market_data', con=engine).head()

In [ ]:
#use pandas to load covid and market data into database
covid_market_df.to_sql(name = 'covid_market_data', con = engine, index_label = 'Date', if_exists = 'replace')
#query tables to validate load
pd.read_sql_query('select * from covid_market_data', con=engine).head()

In [ ]:
#check tables
engine.table_names()

# Extract and Visualize

In [ ]:
#define function for visualization (line plots)
def plotify(pct_kill_rate,market_price):
    pct_kill_rate.plot(use_index=True)
    market_price.plot(use_index=True)
    plt.title(f'Graph of {pct_kill_rate.name} vs. {market_price.name} over applicable date range')
    plt.xlabel(pct_kill_rate.name)
    plt.ylabel(market_price.name)
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
#query sars data for use and reset index for timeline
sars_extract = pd.read_sql_query('select * from sars_market_data', con=engine)
sars_extract = sars_extract.set_index('Date')

#use plotify function to visualize
plotify(sars_extract['Pct_Chng_Death'],sars_extract['Pct_Chng_Market'])

In [ ]:
#query h1n1 data for use and reset index for timeline
h1n1_extract = pd.read_sql_query('select * from h1n1_market_data', con=engine)
h1n1_extract = h1n1_extract.set_index('Date')

#use plotify function to visualize
plotify(h1n1_extract['Pct_Chng_Death'],h1n1_extract['Pct_Chng_Market'])

In [ ]:
#query ebola data for use and reset index for timeline
ebola_extract = pd.read_sql_query('select * from ebola_market_data', con=engine)
ebola_extract = ebola_extract.set_index('Date')

#use plotify function to visualize
plotify(ebola_extract['Pct_Chng_Death'],ebola_extract['Pct_Chng_Market'])

In [ ]:
#query covid data for use and reset index for timeline
covid_extract = pd.read_sql_query('select * from covid_market_data', con=engine)
covid_extract = covid_extract.set_index('Date')

#use plotify function to visualize
plotify(covid_extract['Pct_Chng_Death'],covid_extract['Pct_Chng_Market'])